In [ ]:
import ipywidgets as widgets
from IPython.display import display
import os

box_layout = widgets.Layout(
    display="flex",
    flex_flow="column",
    align_items="center",
    border="1px solid #E0E0E0",
    width="400px",
    padding="25px",
    border_radius="10px"
)

button_layout = widgets.Layout(width="100%", height="40px")

upload_layout = widgets.Layout(width="100%")

title = widgets.HTML(
    "<h3 style='margin-bottom:10px;'>Upload CSV Files 👇</h3>"
)

subtitle = widgets.HTML(
    "<p style='color:gray; margin-top:0;'>Select one or more CSV files to upload and save</p>"
)

train_uploader = widgets.FileUpload(
    accept='.csv',
    multiple=True,
    layout=upload_layout
)

save_button = widgets.Button(
    description='Save Files',
    button_style='primary',
    icon='save',
    layout=button_layout
)

status = widgets.HTML("<p style='color:gray;'>Waiting for upload...</p>")

output = widgets.Output()

def save_files(change=None):
    with output:
        output.clear_output()
        
        if train_uploader.value:
            saved_files = []
            for filename, file_info in train_uploader.value.items():
                with open(filename, "wb") as f:
                    f.write(file_info['content'])
                saved_files.append(filename)
            
            status.value = f"<p style='color:green;'>Saved {len(saved_files)} file(s) 👍</p>"
        else:
            status.value = "<p style='color:red;'>⚠️ No files uploaded yet</p>"

train_uploader.observe(save_files, names='value')

save_button.on_click(save_files)

card = widgets.VBox(
    [title, subtitle, train_uploader, save_button, status],
    layout=box_layout
)

display(card)
display(output)

Output()

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv('file.csv', usecols=['Age', 'Pclass', 'Fare', 'Survived'])

In [4]:
df.sample(2)

,Survived,Pclass,Age,Fare
280,0,3,65.0,7.75
33,0,2,66.0,10.50


In [5]:
df.isnull().sum()

,0
Survived,0
Pclass,0
Age,177
Fare,0


In [7]:
x = df.drop(columns=['Survived'])
y = df['Survived']

In [8]:
xtrain, xtest, ytrain, ytest = train_test_split(
    x,
    y,
    test_size=0.2,
    random_state=42
)

In [14]:
knn = KNNImputer(n_neighbors=5)

xtrain_transform = knn.fit_transform(xtrain)
xtest_transform = knn.transform(xtest)

### **Results**

In [15]:
lr = LogisticRegression()

lr.fit(xtrain_transform, ytrain)
ypred = lr.predict(xtest_transform)
accuracy_score(ytest, ypred)

0.7430167597765364

In [16]:
si = SimpleImputer()

xtrain_si = si.fit_transform(xtrain)
xtest_si = si.transform(xtest)

lr.fit(xtrain_si, ytrain)
ypred_si = lr.predict(xtest_si)
accuracy_score(ytest, ypred_si)

0.7374301675977654

### **Conclusions**

1. knn imputation gives much better results than other imputations


2. It is time taking for large dataset


3. knn imputation is memory heavy - as it needs whole training data set to be available on server during production